In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from selenium import webdriver
import undetected_chromedriver as uc
from time import sleep

# full details

##  get link

In [96]:
#---------------func to get all link----------
def getlink(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("window-size=1920,1080")
    chrome_options.headless = True
    driver = uc.Chrome(options=chrome_options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    
    for text in soup.find_all('a', class_="re__clearfix"):
        house = {'url_parent': url}
        house['url'] = "https://batdongsan.com.vn" + text['href']
        bds.append(house)
#------------------------loop in page list--------------
bds=[]
start = 1
end = 21
print(str(datetime.now()))

i=start
e=[]

while True:
    try:
        if i<end:
            getlink('https://batdongsan.com.vn/du-an-can-ho-chung-cu/p' + str(i))
            # if i % 20 == 0:
            #     print("done index ", i, "at ", datetime.now())
            if (i+1) % 21 == 0:
                print("save================ ", i, "at ", (str(datetime.now())))
                store_file = 'df_link_'+str(start)+'.csv'
                df_link = pd.DataFrame(bds)
                df_link.to_csv(store_file,index = False)
            i=i+1
        else:
            break
    except Exception as ex:
        e.append(i)
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        print(i,"Oops! Meet error,sleep 10s and pass to========= ",i+1)
        i=i+1       
        sleep(10)       

df_link = pd.DataFrame(bds)
df_link['URL_index']=df_link.index
#df_link=pd.read_csv('df_link_all.csv')
url_list = df_link['url'].tolist()
print(len(url_list))

2023-06-07 15:47:01.640033
save================  20 at  2023-06-07 15:53:42.289706
200


## func get data from url

In [194]:
def generate(url):
    # function to get data from each url
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("window-size=1920,1080")
    chrome_options.headless = True
    driver = uc.Chrome(options=chrome_options, version_main=114)
    driver.set_page_load_timeout(60)
    driver.get(url)
    card = BeautifulSoup(driver.page_source)
    house = {'url': url}
    #-------------------------------main content-----------------------------------#
    for item in card.select('.re__project-main-content'):
        house['tongquan']= item.find_all('div', class_="js__prj-detail-content re__detail-content re__project-editor")

    #-------------------------------thong tin chi tiet-------------------------------------#
    for item in card.select('.re__project-box-item'):
        house[item.label.text.strip()] = item.span.text.strip()

    #-------------------------------chu dau tu-------------------------------------#
    for item in card.select('.re__inves-info-row'):
        house[card.find_all('div', class_='re__inves-info-header')[0].text.strip()] = [i.text.strip() for i in card.find_all('div', class_='re__inves-info-cell')]

    #-------------------------------mat bang du an-------------------------------------#
    for item in card.select('.re__full-image'):
        title = item['title']
        src = item['src']
        house['matbangduan'] = f'{title}: {src}'
    
    #-------------------------------photos-------------------------------------#
    for item in card.select('.re__project-main-left'):
        for ele in item.find_all('figure', class_ = 'image'):
            # print(ele)
            if ele.figcaption != None:
                house[ele.figcaption.text.strip()] = ele.img['src']
            else:
                house[ele.figcaption] = ele.img['src']


    # for text in card.select(".re__project-title"):# ten du an neu co
    #     house['project'] = text.text.strip()
    # for text in card.select("#product-detail-web > h1"):
    #     house['name'] = text.text.strip()
    # for text in card.select("#product-detail-web > span"):
    #     house['address'] = text.text.strip()
    # for item in card.select(".re__pr-short-info-item"):
    #     house_list = item.find_all('span')
    #     house[house_list[0].text] = house_list[1].text.strip()
    # for item in card.select(".re__pr-specs-product-type"):
    #     house['house_type'] = item.text.strip()
    # for item in card.select(".re__pr-specs-content-item"):
    #     house_list = item.find_all('span')
    #     house[house_list[1].text] = house_list[2].text.strip()
    
    house['Processed_date'] = str(datetime.now())

    return house

## loop in url list

In [ ]:
%%time
#url_list = df_link['url'].tolist()
print(len(url_list))

start = 0

end = len(url_list)

house_data = []
print(str(datetime.now()))

# loop through url list to extract data 
# starting from 15000
i=start
e=[]
while True:
    #print(i)
    try:
        if i<end:
            #xoa_uc()
            new_house = generate(url_list[i])
            new_house['URL_index'] = i
            house_data.append(new_house)
            
            if i % 10 == 0:
                print("done index ", i, "at ", datetime.now())
            if (i+1) % 50 == 0:
                print("save================ ", i, "at ", (str(datetime.now())))
                store_file = 'df_nharieng_'+str(start)+'.csv'
                df = pd.DataFrame(house_data)
                df.to_csv(store_file,index = False)
            sleep(2)
            i=i+1
        else:
            break

    except Exception as ex:
        e.append(i)
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        print(i,"Oops! Meet error,sleep 10s and pass to========= ",i+1)
        i=i+1       
        sleep(10)
print(str(datetime.now()))

df = pd.DataFrame(house_data)

## Change columns name to save to database

In [ ]:
df_fin=pd.merge(df,df_link[['url_parent','URL_index']], how='left', left_on=['URL_index'], right_on=['URL_index'])

columns_old=['url_parent','url', 'project', 'name', 'address', 'Mức giá',
       'Diện tích', 'Phòng ngủ', 'Ngày đăng', 'Ngày hết hạn', 'house_type',
       'Loại tin', 'Mã tin', 'Mặt tiền', 'Đường vào',
       'Hướng nhà', 'Hướng ban công', 'Số tầng', 'Số phòng ngủ', 'Số toilet',
       'Pháp lý', 'Nội thất','Processed_date']
df_fin=df_fin[columns_old]

columns_new= ['URL_PARENT', 'URL', 'PROJECT', 'NAME', 'ADDRESS', 'MUC_GIA', 'DIEN_TICH',
        'PHONG_NGU', 'NGAY_DANG', 'NGAY_HET_HAN',
        'HOUSE_TYPE', 'LOAI_TIN', 'MA_TIN', 'MAT_TIEN', 'DUONG_VAO',
        'HUONG_NHA', 'HUONG_BAN_CONG', 'SO_TANG', 'SO_PHONG_NGU', 'SO_TOILET', 'PHAP_LY', 'NOI_THAT','DATE']

df_fin.columns=columns_new

# only project address

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("window-size=1920,1080")
# chrome_options.headless = True
driver = uc.Chrome(options=chrome_options)
house = []

In [3]:
#---------------func to get all link----------
def getlink(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    for card in soup.select('.re__prj-card-info-content'):
        project = [i.text.strip() for i in card.find_all('h3', class_='re__prj-card-title')]
        address = [i.text.strip() for i in card.find_all('div', class_='re__prj-card-location')]
        house.append(project + address)

start = 1
end = 157
print(str(datetime.now()))

i=start

while True:
    try:
        if i<=end:
            getlink('https://batdongsan.com.vn/du-an-can-ho-chung-cu/p' + str(i))
            if (i+1) % 21 == 0:
                print("save================ ", i, "at ", (str(datetime.now())))
                df_link = pd.DataFrame(house)
                df_link.columns = ['project', 'address']
                df_link.to_excel('project.xlsx',index = False)
            i=i+1
            sleep(1.5)
        if i == end:
            df_link = pd.DataFrame(house)
            df_link.columns = ['project', 'address']
            df_link.to_excel('project.xlsx',index = False)
        else:
            break
    except Exception as ex:
        e.append(i)
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        print(i,"Oops! Meet error,sleep 10s and pass to========= ",i+1)
        i=i+1       
        sleep(10)       

2023-08-11 09:05:51.069200


KeyboardInterrupt: 

In [118]:
df_link = pd.read_excel('project.xlsx')
df_link['project'] = df_link.project.str.lower()
df_link['address'] = df_link.address.str.lower()

In [119]:
cmv_dfvn = pd.read_excel('cmv_code.xlsx')
def rmv(x):
    x = str(x)
    x = x.lower()
    x = x.replace(' - ', ' ') 
    #x = x.replace('[0-9]', '') 
    x = x.replace('-', ' ') 
    # x = x.replace('/', ' ') 
    x = x.replace('_', ' ') 
    x = x.replace('*', ' ') 
    x = x.replace('.', ' ') 
    x = x.replace('[', ' ') 
    x = x.replace(']', ' ')
    x = x.replace('  ', ' ')
    x = x.replace('\xa0', ' ')
    x = x.replace('thành phố ', ' ')
    x = x.replace('tỉnh ', ' ')
    x = x.replace('huyện ', ' ')
    x = x.replace('quận ', ' ')
    x = x.replace('thị xã ', ' ')
    x = x.replace('xã ', ' ')
    x = x.replace('xã ', ' ')
    x = x.replace('xã ', ' ')
    x = x.replace('phường ', ' ')
    x = x.replace('thị trấn ', ' ')
    x = x.replace('phường ', ' ')
    x = x.strip()
    if x.isnumeric():
        x = str(int(x))
    return x

cmv_dfvn['city'] = cmv_dfvn.TINH.apply(lambda x: rmv(x))
cmv_dfvn['district1'] = cmv_dfvn.HUYEN.apply(lambda x: rmv(x))
cmv_dfvn['ward1'] = cmv_dfvn.XA.apply(lambda x: rmv(x))

cmv_dfvn = cmv_dfvn[['city', 'REF_TINH_CODE', 'district1', 'REF_HUYEN_CODE', 'ward1', 'REF_XA_CODE']]
cmv_dfvn.columns = ['city', 'city_code', 'district', 'district_code', 'ward', 'ward_code']

In [120]:
dfvn = pd.read_excel('dfvn.xlsx')

def clean(x): 
    x = x.lower()
    #x = x.apply(lambda x: unidecode(str(x))) 
    x = x.replace(' - ', ' ') 
    #x = x.replace('[0-9]', '') 
    x = x.replace('-', ' ') 
    # x = x.replace('/', ' ') 
    x = x.replace('_', ' ') 
    x = x.replace('*', ' ') 
    x = x.replace('.', ' ') 
    x = x.replace('[', ' ') 
    x = x.replace(']', ' ')
    x = x.replace('  ', ' ')
    # x = x.replace('tp.', ' ')
    x = x.replace('tp', ' ')
    x = x.replace('hcm', 'hồ chí minh')
    x = x.replace('hà nội', 'hà nội')
    # x = x.replace('oá', 'óa')
    # x = x.replace('oà', 'òa')
    x = x.strip()
    return x

tructhuoc = ['hà nội', 'hồ chí minh', 'cần thơ', 'đà nẵng', 'hải phòng',
            'thành phố hà nội', 'thành phố hồ chí minh', 'thành phố cần thơ', 'thành phố đà nẵng', 'thành phố hải phòng']

def address_sep(x):
    x = clean(x)
    lst = [c.lower().strip() for c in x.split(',')]
    a = {'city':'', 'city_code': -1, 'district':'', 'district_code':-1,'ward':'', 'ward_code':-1, 'road':''}
    road = ''
    if 'việt nam' in lst:
        lst = [i for i in lst if i != 'việt nam']
    tinh = lst[-1]
    if tinh.startswith('tỉnh '):
        a['city']=tinh[5:]
    elif tinh.startswith('thành phố '):
        a['city']=tinh[10:]
    else:
        a['city'] = tinh
    for c in lst:
        if c.startswith('thành phố'):
            if tructhuoc not in lst:
                a['district'] = c[10: ]
        elif c.startswith('quận '):
            a['district'] = c[5:]
        elif c.startswith('huyện '):
            a['district'] = c[6:]
        elif c.startswith('thị xã '):
            a['district'] = c[7:]
        elif c.startswith('thị trấn '):
            a['ward'] = c[10:]
        elif c.startswith('xã '):
            a['ward'] = c[3:]
        elif c.startswith('phường '):
            a['ward'] = c[7:]
        elif c.startswith('đường '):
            a['road'] = c[6:]
            road = c[6:]
        elif c.startswith('phố '):
            a['road'] = c[3:]
            road = c[3:]
    if len(a['city']) < 1 and len(lst) > 0:
        if lst[-1] in dfvn.city.values:
            a['city'] = lst[-1]
    if len(a['district'])<1 and len(lst) > 1:
        if lst[-2] in dfvn[dfvn.city == a['city']].district.values:
            a['district'] = lst[-2]
    if len(a['ward']) <1:
        if len(lst) > 2 and lst[-3] in dfvn[(dfvn.city == a['city']) & (dfvn.district == a['district'])].ward.values:
            a['ward'] = lst[-3]
        if len(lst) > 1 and lst[-2] in dfvn[(dfvn.city == a['city']) & (dfvn.district ==a['district'])].ward.values:
            a['ward'] = lst[-2]
    if 'đường' in lst[0]:
        sindex = lst[0].find('đường')
        a['road'] = lst[0][sindex+6: ]
        road = lst[0][sindex+6: ]
    dfa = pd.DataFrame()
    if len(a['ward']) > 0 and len(a['district']) > 0:
        dfa = cmv_dfvn[(cmv_dfvn.district==a['district']) & (cmv_dfvn.ward==a['ward'])]
    if len(dfa) == 0:
        if len(a['district']) > 0:
            dfa = cmv_dfvn[(cmv_dfvn.city==a['city']) & (cmv_dfvn.district==a['district'])]
    if len(dfa) == 0:
        if len(a['ward']) > 0:
            dfa = cmv_dfvn[(cmv_dfvn.city==a['city']) & (cmv_dfvn.ward==a['ward'])]
    if len(dfa) == 0:
        dfa = cmv_dfvn[cmv_dfvn.city==a['city']]
    if len(dfa) > 0:
        a = dfa.to_dict('records')[0]
        a['road'] = road
        return a
    else: 
        a['road'] = road
        return a

In [121]:
data = []
for i in range(len(df_link)):
    data.append(address_sep(df_link.loc[i, 'address']))

df_new = pd.DataFrame(data)
df_link = pd.concat([df_link, df_new[[i for i in df_new.columns if i != 'project']]], axis=1)

In [122]:
df_link.to_excel('new_project.xlsx', index = False)